# Task 1:  Managing Conversation History with Summarization
1. Maintain a running conversation history of user–assistant chats.
2. Implement summarization of conversation history to keep it concise.
3. Allow customization options for truncation:

          a. Limit by number of conversation turns (e.g., last n messages).

          b. Limit by character/word length.

4. Add periodic summarization:

          a. Perform summarization after every k-th run of the conversation.

          b. Store/replace the summarized version in the conversation history.

5. Demonstrate the functionality by:

          a. Feeding multiple conversation samples.

          b. Showing outputs at different truncation settings.

          c. Showing how summarization happens after every k-th run (e.g., after every 3rd run).

In [ ]:
import os, json
from typing import List, Dict, Any
import requests

In [ ]:
GROQ_API_KEY = "Your Groq api key"

In [ ]:
GROQ_CHAT_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",  # pass API key for authentication
    "Content-Type": "application/json"          # tell server we are sending JSON format
}

call_groq() function is built for sending chat history to Groq API and it give assistant reply for users question.

In [ ]:
def call_groq(messages, model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0.7, max_tokens=5000):

    # Send chat history to Groq API and return assistant reply.

    api_call = {
        "model": model,
        "messages": messages,
        "temperature": temperature,
        "max_tokens": max_tokens
    }
    response = requests.post(GROQ_CHAT_ENDPOINT, headers = HEADERS, json = api_call)
    response.raise_for_status()
    data = response.json()
    return data["choices"][0]["message"]["content"]


summarize_conversation() function summarize whole conversation

In [ ]:
def summarize_conversation(conversation):

    # summarize the whole conversation in a few sentences.

    summary_prompt = conversation + [
        {"role": "user", "content": "Please summarize this conversation in sentences."}
    ]
    return call_groq(summary_prompt)

truncate_conversation() function is used for truncation logic using either last N messages and/or character limit

In [ ]:
def truncate_conversation(conversation, max_turns=None, max_chars=None, method="turns"):

    # Truncate conversation by turns OR by character length.

    if method == "turns" and max_turns is not None:
        return conversation[-max_turns:]   # keep last N messages

    if method == "chars" and max_chars is not None:
        text = ""
        truncated = []
        for msg in reversed(conversation):
            if len(text) + len(msg["content"]) > max_chars:
                break
            truncated.insert(0, msg)
            text += msg["content"]
        return truncated

    return conversation

periodic_summarization() function is for summarizing conversation after k user turns and replace history

In [ ]:
def periodic_summarization(conversation, turn_count, k):

    # Summarize after every k user turns.
    # Replace history with summary as system message.

    if turn_count % k == 0 and turn_count > 0:
        print("\n Summarizing conversation ")
        summary = summarize_conversation(conversation)
        print("Summary:", summary)
        # reset conversation to compact summary
        conversation = [
            {"role": "system", "content": "Running summary: " + summary}
        ]
    return conversation

final_chat() is main function, which gives chance to user for selecting truncation method, then choosing limit and k.
Then take query from user, send it to above function for getting response for that query.

In [ ]:
def final_chat():
    conversation = []
    turn_count = 0

    # User chooses truncation method
    trunc_method = input("Choose truncation method (turns/chars): ").strip().lower()

    max_turns, max_chars = None, None
    if trunc_method == "turns":
        max_turns = int(input("Enter max number of turns to keep: "))
    elif trunc_method == "chars":
        max_chars = int(input("Enter max number of characters to keep: "))

    # User chooses k (summarization frequency)
    k = int(input("Summarize after how many user messages (k)? "))

    print("\nChat started! Type 'exit' to quit.\n")

    while True:
        user_input = input("You: ")

        if user_input.lower() in ["exit", "quit"]:
            print("Chat ended.")
            break

        # Add user message
        conversation.append({"role": "user", "content": user_input})
        turn_count += 1

        # Apply truncation
        conversation = truncate_conversation(conversation, max_turns, max_chars, trunc_method)

        # Call Groq for reply
        reply = call_groq(conversation)
        print("Assistant:", reply)

        # Add assistant reply
        conversation.append({"role": "assistant", "content": reply})

        # Periodic summarization
        conversation = periodic_summarization(conversation, turn_count, k)


# final function
final_chat()

Choose truncation method (turns/chars): turns
Enter max number of turns to keep: 4
Summarize after how many user messages (k)? 3

Chat started! Type 'exit' to quit.

You: what is capital of India
Assistant: The capital of India is **New Delhi**.
You: tell me about blood moon
Assistant: The **Blood Moon**! It's a rare and awe-inspiring lunar event that has captivated humans for centuries. Here's what you need to know:

### **What is a Blood Moon?**  
A Blood Moon occurs during a **total lunar eclipse**, when the Moon passes through Earth's shadow. The Earth's atmosphere scatters shorter wavelengths of light (like blue and violet) more than longer ones (like red and orange). This scattering effect bends some sunlight around the Earth, casting a reddish glow on the Moon, giving it a **rusty, reddish-brown appearance**. 

### **Why "Blood"?**  
The term **"Blood Moon"** isn't scientifically precise but refers to the Moon's reddish hue during the eclipse. This phenomenon has also been calle

# Task 2: JSON Schema Classification & Information Extraction
1. Create a JSON schema to extract 5 details from chats related to information collection (e.g., name, email, phone, location, age).

2. Use OpenAI function calling with Groq API for structured outputs.

3. Demonstrate:

          a. Parsing of at least 3 sample chats.

          b. Validation of outputs against the schema.

In [ ]:
import requests
import json
from jsonschema import validate, ValidationError

# API setup (same as Task 1)
GROQ_API_KEY = "your groq api key"
GROQ_CHAT_ENDPOINT = "https://api.groq.com/openai/v1/chat/completions"
HEADERS = {
    "Authorization": f"Bearer {GROQ_API_KEY}",
    "Content-Type": "application/json"
}

In [ ]:
extraction_schema = {
    "name": "user_info_schema",
    "schema": {
        "type": "object",
        "properties": {
            "name": {"type": "string"},
            "email": {"type": "string"},
            "phone": {"type": "string"},
            "location": {"type": "string"},
            "age": {"type": "integer"}
        },
        "required": ["name", "email", "phone", "location", "age"],
        "additionalProperties": False
    }
}



In [ ]:
functions = [
    {
        "name": "extract_user_info",
        "description": "Extract user details like name, email, phone, location, and age",
        "parameters": {
            "type": "object",
            "properties": {
                "name": {"type": "string"},
                "email": {"type": "string"},
                "phone": {"type": "string"},
                "location": {"type": "string"},
                "age": {"type": "integer"}
            },
            "required": ["name", "email", "phone", "location", "age"]
        }
    }
]


extract_info_from_chat() is final function which extract info. like name, email, phone, location, age from chat

In [ ]:
def extract_info_from_chat(chat_text):
    url = "https://api.groq.com/openai/v1/chat/completions"
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",  # replace with actual key
        "Content-Type": "application/json"
    }

    api_call = {
        "model": "meta-llama/llama-4-scout-17b-16e-instruct",
        "messages": [
            {"role": "system", "content": "Extract user details (name, email, phone, location, age) from the conversation."},
            {"role": "user", "content": chat_text}
        ],
        "response_format": {
            "type": "json_schema",
            "json_schema": extraction_schema
        }
    }

    response = requests.post(url, headers = headers, json = api_call)
    response.raise_for_status()
    data = response.json()

    # Parse structured function call output
    try:
        extracted = json.loads(data["choices"][0]["message"]["content"])
    except KeyError:
        raise ValueError("Unexpected response format. Check API output.")

    # Validate against schema
    try:
        validate(instance = extracted, schema = extraction_schema)
        print("Validation successful:", extracted)
    except ValidationError as e:
        print("Validation failed:", e.message)

    return extracted



In [ ]:
sample_chats = [
    "Hello, This Garima Verma, 22 years old, from Alwar, Rajasthan. My email is garimaverma764@gmail.com or call me on 4567812345.",
    "Hi, my name is Rahul Sharma. I live in Delhi. You can reach me at rahul.sharma@example.com or call me at +91-9876543210. I am 29 years old.",
    "Hello, I’m Sarah Sharma from New Delhi. My email is sarah.s@gmail.com, phone number is 9645386509.",
    "This is Amit Verma, aged 42, from Mumbai. Contact me on amit42@yahoo.com ",
    "My name is Soumya"
]


In [ ]:
for i, chat in enumerate(sample_chats, 1):
    print(f"\n Extracting info from Chat {i} ")
    extract_info_from_chat(chat)


 Extracting info from Chat 1 
Validation successful: {'name': 'Garima Verma', 'email': 'garimaverma764@gmail.com', 'phone': '4567812345', 'location': 'Alwar, Rajasthan', 'age': 22}

 Extracting info from Chat 2 
Validation successful: {'name': 'Rahul Sharma', 'email': 'rahul.sharma@example.com', 'phone': '+91-9876543210', 'location': 'Delhi', 'age': 29}

 Extracting info from Chat 3 
Validation successful: {'name': 'Sarah Sharma', 'email': 'sarah.s@gmail.com', 'phone': '9645386509', 'location': 'New Delhi', 'age': None}

 Extracting info from Chat 4 
Validation successful: {'name': 'Amit Verma', 'email': 'amit42@yahoo.com', 'phone': '', 'location': 'Mumbai', 'age': 42}

 Extracting info from Chat 5 
Validation successful: {'name': 'Soumya'}
